In [2]:
import pandas as pd
import numpy as np
import pymysql

### 1. Data Loading

In [3]:
data=pd.read_csv('Rent_houses_dataset.csv')
df=pd.DataFrame(data)

In [4]:
df.head(3)

,HouseTitle,Builder,Location,Furnishing,Bathroom,TenantPrefered,Availability,Area,Floor,PropertyFacing,Notes,Rent,NearBy,Link
0,"5 BHK Flat for Rent in Hebbal, Bangalore",Embassy Lake Terraces,Not Available,Semi-Furnished,6,Bachelors/Family,Immediately,6300,12 out of 20,Not Available,Nice flat with a good ventilation. The monthly...,430000.0,"1.9 Km from Kirloskar Tech Park, 0.1 Km from U...",https://www.magicbricks.com/propertyDetails/5-...
1,"3 BHK Flat for Rent in Jakkur, Bangalore",Sobha HRC Pristine,Not Available,Semi-Furnished,3,Bachelors/Family,Immediately,1400,17 out of 20,East Facing Property,Premium Property located in one of the prime l...,67000.0,"1.9 Km from Kirloskar Tech Park, 0.1 Km from U...",https://www.magicbricks.com/propertyDetails/5-...
2,"2 BHK Flat for Rent in Hebbal, Bangalore",Prestige Misty Waters,Not Available,Semi-Furnished,2,Bachelors/Family,Immediately,820,12 out of 23,East Facing Property,"Good ventilation and good interiors, well main...",41000.0,"1.9 Km from Kirloskar Tech Park, 0.1 Km from U...",https://www.magicbricks.com/propertyDetails/5-...


In [5]:
df.shape

(1500, 14)

### 2. Data Cleaning

In [6]:
df.isna().sum().any()

False

In [7]:
df.isna().sum()

HouseTitle        0
Builder           0
Location          0
Furnishing        0
Bathroom          0
TenantPrefered    0
Availability      0
Area              0
Floor             0
PropertyFacing    0
Notes             0
Rent              0
NearBy            0
Link              0
dtype: int64

In [139]:
df.rename(columns={'House title': 'HouseTitle'}, inplace=True)
df.rename(columns={'Tenant Prefered': 'TenantPrefered'}, inplace=True)
df.rename(columns={'Property Facing': 'PropertyFacing'}, inplace=True)
df.rename(columns={'Area':'Area_sqft'}, inplace=True)

In [5]:
df['Builder']=df['Builder'].fillna("Not Available")

In [6]:
df['PropertyFacing']=df['PropertyFacing'].fillna("Not Available")

In [7]:
df['Rent']=df['Rent'].str.replace(',', '')
df['Rent']=df['Rent'].str.replace('₹', '')
df['Rent'] = df['Rent'].apply(lambda x: float(x.replace('Lac', '')) * 100000 if 'Lac' in x else float(x))

In [8]:
df['Rent']

0       430000.0
1        67000.0
2        41000.0
3       110000.0
4       180000.0
          ...   
1495     50000.0
1496     20000.0
1497     60000.0
1498    200000.0
1499     47999.0
Name: Rent, Length: 1500, dtype: float64

In [9]:
df['Area']=df['Area'].str.replace('sqft','')

In [10]:
df['Area'].astype('int')

0       6300
1       1400
2        820
3       2470
4       3400
        ... 
1495    1250
1496     940
1497    1655
1498    3105
1499    1600
Name: Area, Length: 1500, dtype: int32

In [11]:
df.columns

Index(['HouseTitle', 'Builder', 'Furnishing', 'Bathroom', 'TenantPrefered',
       'Availability', 'Area', 'Floor', 'PropertyFacing', 'Notes', 'Rent',
       'NearBy', 'Link'],
      dtype='object')

In [162]:
df = df.drop(df.columns[[6]], axis=1)

In [178]:
df.drop(['PropertyFacing.1', 'HouseTitle.1','TenantPrefered.1', 'Area(sqft)'],axis=1)

In [19]:
df['Area'].dtype

dtype('O')

In [20]:
df['Area']=df['Area'].astype('int')

In [21]:
df['Rent'].dtype

dtype('float64')

In [12]:
df_split=df['HouseTitle'].str.split(" ",expand=True)

In [13]:
df_split=df_split.drop([0,1,2,3,4,5],axis=1)

In [14]:
df_split

,6,7,8,9,10
0,"Hebbal,",Bangalore,None,None,None
1,"Jakkur,",Bangalore,None,None,None
2,"Hebbal,",Bangalore,None,None,None
3,"Hebbal,",Bangalore,None,None,None
4,"Hebbal,",Bangalore,None,None,None
...,...,...,...,...,...
1495,Ecc,"Road,","Whitefield,",Bangalore,None
1496,"Rajapura,",Bangalore,None,None,None
1497,"Whitefield,",Bangalore,None,None,None
1498,Whitefield,Bangalore,None,None,None


In [15]:
df2=df_split[6]+df_split[7]+df_split[8]+df_split[9]+df_split[10]

In [16]:
df2=df2.fillna('Not Available')

In [17]:
df2=df2.replace('Phase7JPNagar,Bangalore','JPNagar,Bangalore')

In [18]:
df2=df2.replace('RichardsTown,RichardsTown,Bangalore','RichardsTown,Bangalore')

In [19]:
df2=df2.replace('RichmondTown,RichmondTown,Bangalore','RichmondTown,Bangalore')

In [20]:
df2.value_counts()

Not Available             1350
RichardsTown,Bangalore      50
RichmondTown,Bangalore      50
JPNagar,Bangalore           50
dtype: int64

In [21]:
for i in df2:
    print(i)

Not Available
Not Available
Not Available
Not Available
Not Available
Not Available
Not Available
Not Available
Not Available
Not Available
Not Available
Not Available
Not Available
Not Available
Not Available
RichardsTown,Bangalore
Not Available
Not Available
RichmondTown,Bangalore
Not Available
Not Available
Not Available
Not Available
JPNagar,Bangalore
Not Available
Not Available
Not Available
Not Available
Not Available
Not Available
Not Available
Not Available
Not Available
Not Available
Not Available
Not Available
Not Available
Not Available
Not Available
Not Available
Not Available
Not Available
Not Available
Not Available
Not Available
RichardsTown,Bangalore
Not Available
Not Available
RichmondTown,Bangalore
Not Available
Not Available
Not Available
Not Available
JPNagar,Bangalore
Not Available
Not Available
Not Available
Not Available
Not Available
Not Available
Not Available
Not Available
Not Available
Not Available
Not Available
Not Available
Not Available
Not Available
Not 

In [22]:
df.insert(13,"Location",df2,True)

In [36]:
df.columns

Index(['HouseTitle', 'Builder', 'Location', 'Furnishing', 'Bathroom',
       'TenantPrefered', 'Availability', 'Area', 'Floor', 'PropertyFacing',
       'Notes', 'Rent', 'NearBy', 'Link'],
      dtype='object')

In [24]:
df=df[['HouseTitle', 'Builder', 'Location', 'Furnishing', 'Bathroom', 'TenantPrefered',
       'Availability', 'Area', 'Floor', 'PropertyFacing', 'Notes', 'Rent', 'NearBy',
       'Link']]

In [29]:
df['Rent']

0       430000.0
1        67000.0
2        41000.0
3       110000.0
4       180000.0
          ...   
1495     50000.0
1496     20000.0
1497     60000.0
1498    200000.0
1499     47999.0
Name: Rent, Length: 1500, dtype: float64

In [37]:
df.to_csv('Rent_houses_dataset.csv',index=False)

### 3. SQL Database Connection

In [8]:
!pip install pymysql

In [11]:
!pip install --upgrade pip

In [10]:
import pymysql

# Establish a database connection
connection = pymysql.connect(
    host='127.0.0.1',
    port=3306,
    user='root',
    password='password',
    db='Real_Estate_Market'
)

# Create a cursor
cursor = connection.cursor()

# Define your CREATE TABLE query
create_table_query = """
CREATE TABLE IF NOT EXISTS Houses (
    id INT AUTO_INCREMENT PRIMARY KEY,
    HouseTitle VARCHAR(255),
    Builder VARCHAR(255),
    Location VARCHAR(255),  # Make sure this matches your dataframe column name
    Furnishing VARCHAR(255),
    Bathroom INT,
    TenantPrefered VARCHAR(255),
    Availability VARCHAR(255),
    Area VARCHAR(255),
    Floor VARCHAR(255),
    PropertyFacing VARCHAR(255),
    Notes TEXT,
    Rent VARCHAR(255),
    NearBy VARCHAR(255),
    Link VARCHAR(255)
)
"""

# Execute the CREATE TABLE query
cursor.execute(create_table_query)

# Iterate over your dataframe and insert data
for _, row in df.iterrows():
    insert_query = """
    INSERT INTO Houses (HouseTitle, Builder, Location, Furnishing, Bathroom, TenantPrefered, Availability, Area, Floor, PropertyFacing, Notes, Rent, NearBy, Link)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    cursor.execute(insert_query, (
        row['HouseTitle'],
        row['Builder'],
        row['Location'],  
        row['Furnishing'],
        row['Bathroom'],
        row['TenantPrefered'],
        row['Availability'],
        row['Area'],
        row['Floor'],
        row['PropertyFacing'],
        row['Notes'],
        row['Rent'],
        row['NearBy'],
        row['Link']
    ))

# Commit changes
connection.commit()


In [8]:
df.columns

Index(['HouseTitle', 'Builder', 'Location', 'Furnishing', 'Bathroom',
       'TenantPrefered', 'Availability', 'Area', 'Floor', 'PropertyFacing',
       'Notes', 'Rent', 'NearBy', 'Link'],
      dtype='object')

In [9]:
df.dtypes

HouseTitle         object
Builder            object
Location           object
Furnishing         object
Bathroom            int64
TenantPrefered     object
Availability       object
Area                int64
Floor              object
PropertyFacing     object
Notes              object
Rent              float64
NearBy             object
Link               object
dtype: object